# Lab — Logistic Regression 


## 0) Setup
Run the next cell first.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, log_loss
from sklearn.linear_model import LogisticRegression

np.random.seed(42)


## 1) Binary classification + simulated data 

In the lecture:
- labels are **0** (negative class) and **1** (positive class)
- we want to estimate the probability $P(y=1\mid x)$

We simulate a 2D dataset so we can plot:
- the points
- the decision boundary
- how the model changes during training


In [ ]:

X, y = make_blobs(
    n_samples=400,
    centers=2,
    n_features=2,
    cluster_std=2.0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Standardize features (fit only on training data)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)


In [ ]:

def plot_data(X, y, title=""):
    plt.figure(figsize=(6, 5))
    plt.scatter(X[y==0, 0], X[y==0, 1], alpha=0.7, label="Class 0")
    plt.scatter(X[y==1, 0], X[y==1, 1], alpha=0.7, label="Class 1")
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_data(X_train_s, y_train, title="Training data (standardized)")


## 2) Model: hypothesis + sigmoid

### Linear score
We first compute a linear combination:

$$
z = \theta_0 + \theta_1 x_1 + \theta_2 x_2
$$

### Logistic hypothesis
Then we pass it through the sigmoid function:

$$
h_\theta(x) = \sigma(z) = \frac{1}{1+e^{-z}}
$$

### Probabilistic interpretation
We interpret:

$$
h_\theta(x) = P(y=1\mid x)
$$

and therefore:

$$
P(y=0\mid x) = 1 - h_\theta(x)
$$

### Useful calculus fact 
Derivative of sigmoid:

$$
\sigma'(z) = \sigma(z)(1-\sigma(z))
$$


## 3) Cost function (Log Loss / Negative Log-Likelihood)



For a single sample:
- If $y=1$: cost is $-\log(h_\theta(x))$
- If $y=0$: cost is $-\log(1-h_\theta(x))$

Combined into one expression:

$$
\text{cost}(h_\theta(x), y) = -\left[y\log(h_\theta(x)) + (1-y)\log(1-h_\theta(x))\right]
$$

For the dataset (mean cost):

$$
J(\theta) = \frac{1}{m}\sum_{i=1}^{m} \text{cost}(h_\theta(x^{(i)}), y^{(i)})
$$

We'll implement this **as log loss** in code.


### 3.1) TODO — implement the key building blocks

Fill in:
- `sigmoid(z)`
- `predict_proba(X, theta)`  (returns $h_\theta(x)$ for each row)
- `log_loss_cost(y, p)`      (mean log loss / NLL)

**(For extra numerical stability):**
Use `np.clip(p, eps, 1-eps)` to avoid `log(0)`.


In [ ]:

# ============================================================
# TODO (STUDENTS)
# ============================================================

def sigmoid(z):
    """sigma(z) = 1/(1+exp(-z))"""
    # TODO
    pass

def predict_proba(X, theta):
    """Return h_theta(x) for each row in X.
    Convention: theta[0] is the bias term (intercept).
    """
    # TODO
    pass

def log_loss_cost(y_true, p_pred, eps=1e-12):
    """Mean log loss / negative log-likelihood."""
    # TODO
    pass


## 4) Gradient Descent 

In the lecture, the update was shown as:

$$
\theta_j := \theta_j - \alpha \cdot \frac{1}{m}\sum_{i=1}^{m}\left(h_\theta(x^{(i)}) - y^{(i)}\right)x_j^{(i)}
$$

and the intercept term (bias) updates similarly (with $x_0^{(i)}=1$).

### TODO
Implement the gradient step using the formula above.


In [ ]:

# ============================================================
# TODO (STUDENTS)
# ============================================================

def fit_logreg_gd(X, y, lr=0.2, n_steps=250):
    """Gradient descent for logistic regression.
    theta[0] is bias, theta[1:] are feature weights.
    """
    m, d = X.shape
    theta = np.zeros(d + 1)
    losses = []

    for step in range(n_steps):
        p = predict_proba(X, theta)
        losses.append(log_loss_cost(y, p))

        # TODO: compute gradients according to the slide formula
        # For j=0 (bias): use x0 = 1
        # For j>=1: use the feature column
        pass

    return theta, np.array(losses)


## 5) Train + visualize learning


In [ ]:

theta, losses = fit_logreg_gd(X_train_s, y_train, lr=0.2, n_steps=250)

plt.figure(figsize=(6,4))
plt.plot(losses)
plt.xlabel("Gradient descent step")
plt.ylabel("Log loss / NLL")
plt.title("Training cost J(θ) (should decrease)")
plt.grid(True, alpha=0.3)
plt.show()

theta


## 6) Decision boundary 

Recall the logistic-regression prediction rule with threshold 0.5:

$$
\hat{y}=1 \quad \text{if} \quad h_\theta(x) \ge 0.5
$$

Because the sigmoid satisfies $\sigma(z)=0.5$ exactly when $z=0$, the **decision boundary** is the set of points where the linear score is zero:

$$
z = \theta_0 + \theta_1 x_1 + \theta_2 x_2 = 0
$$

That equation describes a **line** in the $(x_1, x_2)$ plane. To plot it, we typically solve for $x_2$ as a function of $x_1$:

$$
\theta_0 + \theta_1 x_1 + \theta_2 x_2 = 0
\;\;\Rightarrow\;\;
x_2 = -\frac{\theta_0 + \theta_1 x_1}{\theta_2}
$$

### Your task
Implement a function:

- **Name:** `decision_boundary_y(x1, theta)`
- **Input:** 
  - `x1`: a NumPy array of $x_1$ values
  - `theta`: a length-3 vector $[\theta_0,\theta_1,\theta_2]$
- **Output:** 
  - a NumPy array with the corresponding boundary values $x_2$ computed from the formula above.

⚠️ Edge case: if $\theta_2 \approx 0$, the boundary is (nearly) a **vertical line** and the formula above would divide by zero.  
In that case, return something safe (e.g. `np.zeros_like(x1)`) so the code runs, and leave a short comment explaining why.


In [ ]:
# TODO: implement the decision boundary function based on the derivation above.
def decision_boundary_y(x1, theta):
    """
    Return x2 values for the decision boundary z=0:
        theta0 + theta1*x1 + theta2*x2 = 0  ->  x2 = -(theta0 + theta1*x1)/theta2

    Parameters
    ----------
    x1 : np.ndarray
        Array of x1 values (already in the same feature space as training data, e.g. scaled).
    theta : array-like, shape (3,)
        [theta0, theta1, theta2]

    Returns
    -------
    x2 : np.ndarray
        Boundary x2 values for each x1.
    """
    # --- STUDENT TODO START ---
    raise NotImplementedError("Implement decision_boundary_y(x1, theta)")
    # --- STUDENT TODO END ---

plt.figure(figsize=(6, 5))
plt.scatter(X_train_s[y_train==0, 0], X_train_s[y_train==0, 1], alpha=0.35, label="Class 0")
plt.scatter(X_train_s[y_train==1, 0], X_train_s[y_train==1, 1], alpha=0.35, label="Class 1")

x1 = np.linspace(X_train_s[:, 0].min() - 1, X_train_s[:, 0].max() + 1, 200)
plt.plot(x1, decision_boundary_y(x1, theta), linewidth=3, label="Decision boundary (p=0.5)")

plt.title("Decision boundary (from scratch)")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


## 8) Evaluate on test set (accuracy + confusion matrix + probability histogram)


In [ ]:

p_test = predict_proba(X_test_s, theta)
y_pred = (p_test >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
print("Test accuracy:", acc)
print("Test log loss:", log_loss(y_test, p_test))

cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot()
plt.title(f"Confusion Matrix (from scratch) — acc={acc:.3f}")
plt.show()

plt.figure(figsize=(6,4))
plt.hist(p_test[y_test==0], bins=20, alpha=0.7, label="True class 0")
plt.hist(p_test[y_test==1], bins=20, alpha=0.7, label="True class 1")
plt.xlabel("Predicted probability $h_\theta(x)$")
plt.ylabel("Count")
plt.title("Predicted probabilities on test set")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## 9) Regularization (L2 and L1)

To reduce overfitting, we modify the objective function by adding a penalty term
that discourages large parameter values.

Recall the gradient descent update rule:

$$
\theta_j :=
\theta_j
-
\alpha
\frac{\partial J(\theta)}{\partial \theta_j}
$$

---

### L2 regularization (Ridge)

We define the regularized objective:

$$
J_{reg}(\theta)
=
J(\theta)
+
\lambda
\sum_{j=1}^{n} \theta_j^2
$$

The update rule becomes:

$$
\theta_j :=
\theta_j
-
\alpha
\left(
\frac{\partial J(\theta)}{\partial \theta_j}
+
2\lambda\theta_j
\right),
\quad j \ge 1
$$

---

### L1 regularization (Lasso)

$$
J_{reg}(\theta)
=
J(\theta)
+
\lambda
\sum_{j=1}^{n} |\theta_j|
$$

Using a sub-gradient:

$$
\theta_j :=
\theta_j
-
\alpha
\left(
\frac{\partial J(\theta)}{\partial \theta_j}
+
\lambda\,\text{sign}(\theta_j)
\right),
\quad j \ge 1
$$

---

> The bias term $\theta_0$ is typically **not regularized**.


### Dataset where regularization actually helps

We keep the 2 real features (so the boundary is plottable), but we add many **noise features**.
Without regularization, the model can overfit by giving large weights to noise features.


In [ ]:

# Build a "harder" version: 2 real features + many noise features
X2, y2 = make_blobs(
    n_samples=500, centers=2, n_features=2, cluster_std=2.2, random_state=7
)
n_noise = 20
noise = np.random.normal(size=(X2.shape[0], n_noise))
Xn = np.hstack([X2, noise])

Xn_train, Xn_test, yn_train, yn_test = train_test_split(
    Xn, y2, test_size=0.3, random_state=42, stratify=y2
)

scaler2 = StandardScaler()
Xn_train_s = scaler2.fit_transform(Xn_train)
Xn_test_s  = scaler2.transform(Xn_test)

plot_data(Xn_train_s[:, :2], yn_train, title="Training data (first two features) + hidden noise features")


### TODO — implement regularized gradient descent

Keep the same structure as before, but add the extra gradient term:
- `reg=None`
- `reg="l2"`
- `reg="l1"`

Remember: **do not regularize** the bias term `theta[0]`.


In [ ]:

# ============================================================
# TODO: Implement gradient descent with regularization (L2 and L1) 
# ============================================================

def fit_logreg_gd_regularized(X, y, lr=0.2, n_steps=250, reg=None, lam=0.1):
    m, d = X.shape
    theta = np.zeros(d + 1)
    losses = []

    for step in range(n_steps):
        p = predict_proba(X, theta)
        losses.append(log_loss_cost(y, p))

        diff = (p - y)
        grad0 = np.mean(diff)
        grad_rest = (X.T @ diff) / m

        # =======================================================
        # TODO: add regularization 
        # =======================================================


    return theta, np.array(losses)


### Evaluate and compare regularization 

We will compare **No regularization vs L2 vs L1** using:
- **Accuracy** (based on hard predictions $\hat{y}$)
- **Log loss** (based on probabilities $p$)

#### Your task (mandatory)
In the function `eval_theta(Xt, yt, theta)` you must compute:

1) **Predicted probabilities**  

$$
p = h_\theta(x) \in [0,1]
$$

2) **Hard class predictions**  

$$
\hat{y} =
\begin{cases}
1 & \text{if } p \ge 0.5 \\
0 & \text{otherwise}
\end{cases}
$$

So in code:
- `p` must be a NumPy array of probabilities
- `yhat` must be a NumPy array of integers (0/1)

Only after you compute `p` and `yhat` should you return:
- `accuracy_score(yt, yhat)`
- `log_loss(yt, p)`



In [ ]:

theta_none, loss_none = fit_logreg_gd_regularized(Xn_train_s, yn_train, reg=None, lam=0.1, lr=0.2, n_steps=300)
theta_l2,   loss_l2   = fit_logreg_gd_regularized(Xn_train_s, yn_train, reg="l2", lam=0.1, lr=0.2, n_steps=300)
theta_l1,   loss_l1   = fit_logreg_gd_regularized(Xn_train_s, yn_train, reg="l1", lam=0.05, lr=0.2, n_steps=300)


# ============================================================
# TODO (STUDENTS): implement p and yhat
# ============================================================
def eval_theta(Xt, yt, theta):
    """
    Evaluate a trained parameter vector theta on a test set.

    YOU must implement:
      - p: predicted probabilities
      - yhat: predicted labels using threshold 0.5

    Returns:
        - accuracy score
        - log loss (NLL)
    """
    # --- STUDENT TODO START ---
    # 1) p = ...
    # 2) yhat = ...
    # return accuracy_score(yt, yhat), log_loss(yt, p)
    raise NotImplementedError("Compute p and yhat in eval_theta()")
    # --- STUDENT TODO END ---



acc_none, ll_none = eval_theta(Xn_test_s, yn_test, theta_none)
acc_l2,   ll_l2   = eval_theta(Xn_test_s, yn_test, theta_l2)
acc_l1,   ll_l1   = eval_theta(Xn_test_s, yn_test, theta_l1)

fig = plt.figure(figsize=(15,4))

#----------- (1) cost curves----------
ax1 = fig.add_subplot(1,3,1)
ax1.plot(loss_none, label=f"No reg (acc={acc_none:.3f})")
ax1.plot(loss_l2,   label=f"L2 (acc={acc_l2:.3f})")
ax1.plot(loss_l1,   label=f"L1 (acc={acc_l1:.3f})")
ax1.set_title("Training cost J(θ)")
ax1.set_xlabel("GD step")
ax1.set_ylabel("Log loss / NLL")
ax1.grid(True, alpha=0.3)
ax1.legend()

#---------- (2) boundaries (first two features)----------
ax2 = fig.add_subplot(1,3,2)

X2_train = Xn_train_s[:, :2]
ax2.scatter(X2_train[yn_train==0,0], X2_train[yn_train==0,1], alpha=0.35, label="Class 0")
ax2.scatter(X2_train[yn_train==1,0], X2_train[yn_train==1,1], alpha=0.35, label="Class 1")

# Compute axis limits from the data (with padding)
x_min, x_max = X2_train[:,0].min(), X2_train[:,0].max()
y_min, y_max = X2_train[:,1].min(), X2_train[:,1].max()
pad_x = 0.10 * (x_max - x_min + 1e-12)
pad_y = 0.10 * (y_max - y_min + 1e-12)

ax2.set_xlim(x_min - pad_x, x_max + pad_x)
ax2.set_ylim(y_min - pad_y, y_max + pad_y)

x1 = np.linspace(x_min - pad_x, x_max + pad_x, 300)

def plot_boundary(ax, theta, label, **kwargs):
    # If theta2 is ~0 -> boundary is (approximately) vertical: theta0 + theta1*x1 = 0
    if abs(theta[2]) < 1e-6:
        if abs(theta[1]) < 1e-12:
            return  # degenerate
        x1_star = -theta[0] / theta[1]
        ax.axvline(x1_star, label=label, **kwargs)
        return

    x2 = -(theta[0] + theta[1]*x1) / theta[2]

    # Clip the line to the visible y-range so it can't blow up the plot
    y_lo, y_hi = ax.get_ylim()
    x2_clipped = np.clip(x2, y_lo, y_hi)

    ax.plot(x1, x2_clipped, label=label, **kwargs)

plot_boundary(ax2, theta_none, "No reg", linewidth=3)
plot_boundary(ax2, theta_l2,   "L2", linestyle="--", linewidth=2)
plot_boundary(ax2, theta_l1,   "L1", linestyle=":", linewidth=3)

ax2.set_title("Decision boundary (first 2 features)")
ax2.set_xlabel("$x_1$")
ax2.set_ylabel("$x_2$")
ax2.grid(True, alpha=0.3)
ax2.legend()

#------------- (3) weight magnitudes (excluding bias)--------------------
ax3 = fig.add_subplot(1,3,3)
ax3.hist(np.abs(theta_none[1:]), bins=20, alpha=0.6, label="No reg")
ax3.hist(np.abs(theta_l2[1:]), bins=20, alpha=0.6, label="L2")
ax3.hist(np.abs(theta_l1[1:]), bins=20, alpha=0.6, label="L1")
ax3.set_title("Distribution of |weights| (excluding bias)")
ax3.set_xlabel("|θ_j|")
ax3.set_ylabel("Count")
ax3.grid(True, alpha=0.3)
ax3.legend()

plt.show()

print("Test metrics (noise-features dataset):")
print(f"No reg: acc={acc_none:.3f}, log-loss={ll_none:.3f}")
print(f"L2    : acc={acc_l2:.3f}, log-loss={ll_l2:.3f}")
print(f"L1    : acc={acc_l1:.3f}, log-loss={ll_l1:.3f}")


## 10) Sanity-check with sklearn

Sklearn is used only as a check that the boundary is consistent.


In [ ]:
# ------------------------------------------------------------
# Train sklearn on SAME dataset as our from-scratch model
# ------------------------------------------------------------
clf = LogisticRegression(penalty="l2", solver="lbfgs", max_iter=2000)
clf.fit(Xn_train_s, yn_train)
theta_sk = np.r_[clf.intercept_.item(), clf.coef_.ravel()]

# ------------------------------------------------------------
# Compute test metrics (no plotting here)
# ------------------------------------------------------------
# From scratch (L2 model)
p_scratch = predict_proba(Xn_test_s, theta_l2)
yhat_scratch = (p_scratch >= 0.5).astype(int)
acc_scratch = accuracy_score(yn_test, yhat_scratch)
ll_scratch = log_loss(yn_test, p_scratch)

# sklearn
p_sk = clf.predict_proba(Xn_test_s)[:, 1]
yhat_sk = (p_sk >= 0.5).astype(int)
acc_sk = accuracy_score(yn_test, yhat_sk)
ll_sk = log_loss(yn_test, p_sk)

# ------------------------------------------------------------
# Plot: decision boundary slice only
# ------------------------------------------------------------
plt.figure(figsize=(6, 5))

X2_train = Xn_train_s[:, :2]
plt.scatter(X2_train[yn_train == 0, 0], X2_train[yn_train == 0, 1],
            alpha=0.35, label="Class 0")
plt.scatter(X2_train[yn_train == 1, 0], X2_train[yn_train == 1, 1],
            alpha=0.35, label="Class 1")

# axis limits from data (with padding)
x_min, x_max = X2_train[:, 0].min(), X2_train[:, 0].max()
y_min, y_max = X2_train[:, 1].min(), X2_train[:, 1].max()
pad_x = 0.10 * (x_max - x_min + 1e-12)
pad_y = 0.10 * (y_max - y_min + 1e-12)

ax = plt.gca()
ax.set_xlim(x_min - pad_x, x_max + pad_x)
ax.set_ylim(y_min - pad_y, y_max + pad_y)

x1_grid = np.linspace(x_min - pad_x, x_max + pad_x, 400)

def plot_boundary_slice(ax, theta, x1_grid, label, x_rest=None, **kwargs):
    d = len(theta) - 1
    if x_rest is None:
        x_rest = np.zeros(max(d - 2, 0))

    # Near-vertical boundary
    if abs(theta[2]) < 1e-8:
        denom = theta[1]
        if abs(denom) < 1e-12:
            return
        const = theta[0]
        if d > 2:
            const += np.dot(theta[3:], x_rest)
        x1_star = -const / denom
        ax.axvline(x1_star, label=label, **kwargs)
        return

    const = theta[0] + theta[1] * x1_grid
    if d > 2:
        const = const + np.dot(theta[3:], x_rest)

    x2 = -const / theta[2]

    # Mask outside current y-limits (no clipping artefacts)
    y_lo, y_hi = ax.get_ylim()
    x2_masked = np.where((x2 >= y_lo) & (x2 <= y_hi), x2, np.nan)

    ax.plot(x1_grid, x2_masked, label=label, **kwargs)

plot_boundary_slice(ax, theta_l2, x1_grid,
                    "From scratch (L2 slice)", linewidth=3)
plot_boundary_slice(ax, theta_sk, x1_grid,
                    "sklearn (L2 slice)", linestyle="--", linewidth=2)

plt.title("Decision boundary comparison (2D slice)")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# ------------------------------------------------------------
# Print comparison results clearly
# ------------------------------------------------------------
print("Test metrics on SAME noise-features dataset")
print("------------------------------------------------")
print(f"From scratch (L2):")
print(f"  Accuracy : {acc_scratch:.4f}")
print(f"  Log-loss : {ll_scratch:.4f}")
print()
print(f"sklearn LogisticRegression (L2):")
print(f"  Accuracy : {acc_sk:.4f}")
print(f"  Log-loss : {ll_sk:.4f}")
